In [1]:
import lightgbm as lgb
import numpy as np
import pandas as pd

# from sklearn.metrics import accuracy_score, auc, precision_score, recall_score, roc_auc_score, log_loss
from tqdm.notebook import tqdm
import itertools

# import dask
# from ray import train, tune
# from ray.tune.search.optuna import OptunaSearch
# from ray.tune.s|chedulers import ASHAScheduler
# from ray.tune.search import ConcurrencyLimiter
# from optuna.samplers import TPESampler

In [2]:
fn_train_x = '/Volumes/Extreme SSD/rematch_eia_ferc1_docker/working_data/model_a/train/train_x.parquet'
fn_train_y = '/Volumes/Extreme SSD/rematch_eia_ferc1_docker/working_data/model_a/train/train_y.parquet'

fn_test_x = '/Volumes/Extreme SSD/rematch_eia_ferc1_docker/working_data/model_a/train/test_x.parquet'
fn_test_y = '/Volumes/Extreme SSD/rematch_eia_ferc1_docker/working_data/model_a/train/test_y.parquet'

fn_train_id = '/Volumes/Extreme SSD/rematch_eia_ferc1_docker/working_data/model_a/train/train_id.parquet'
fn_test_id = '/Volumes/Extreme SSD/rematch_eia_ferc1_docker/working_data/model_a/train/test_id.parquet'

fn_grid = '/Volumes/Extreme SSD/rematch_eia_ferc1_docker/working_data/model_a/train/gb_ray_tune/grid_search.csv'

In [3]:
TrainX = pd.read_parquet(fn_train_x)
TrainY = pd.read_parquet(fn_train_y)
TestX = pd.read_parquet(fn_test_x)
TestY = pd.read_parquet(fn_test_y)
TrainID = pd.read_parquet(fn_train_id)
TestID = pd.read_parquet(fn_test_id)

In [4]:
X = pd.concat([TrainX, TestX])
Y = pd.concat([TrainY, TestY])
ID = pd.concat([TrainID, TestID])

In [5]:
rename_dict = {'config/num_trees':'num_trees', 'config/learning_rate':'learning_rate', 'config/min_data_in_leaf':'min_data_in_leaf', 'config/objective':'objective', 'config/early_stopping_round':'early_stopping_round'} #, 'config/metrics':'metrics'}

Grid = pd.read_csv(fn_grid, index_col='rank')
Grid = Grid.rename(columns=rename_dict)[list(rename_dict.values())]

# Create a dictionary: punch in the rank of the model we want to use, and get the parameters back, as a dictionary
param_dict = {i:Grid.loc[i].to_dict() for i in Grid.index}
param_dict[0]

{'num_trees': 437,
 'learning_rate': 0.0076861305642603,
 'min_data_in_leaf': 76,
 'objective': 'binary',
 'early_stopping_round': -1}

In [6]:
# Assign a number, 0-4, for each unique ferc id

unique_ferc_ids = ID['record_id_ferc1'].unique()
num_folds = 5
fold_list = np.random.choice(a=range(num_folds), size=len(unique_ferc_ids))
id_to_fold_dict = dict(zip(unique_ferc_ids.tolist(), fold_list))

list(id_to_fold_dict.items())[:5]

[('f1_steam_2005_12_108_0_1', 1),
 ('f1_steam_2005_12_108_0_2', 0),
 ('f1_steam_2005_12_108_0_4', 2),
 ('f1_steam_2005_12_108_0_5', 0),
 ('f1_steam_2005_12_108_1_1', 4)]

Each record_id_ferc1 is mapped to a number, 0:num_folds. 

This means that for any fold, we can say that any record_id_ferc1 that corresponds to that number is testing data, and all others are training data

In [8]:
variables = [
    range(2),  # num hyperparameters to test
    range(2)
    # range(num_folds)  # number of folds per test
]

y_fit_list = []
for (hp_rank, fold) in tqdm(list(itertools.product(*variables))):

    # Split into training and test sets, based on the id_to_fold_dict mapping to the ID array
    is_training_data_mask = [ id_to_fold_dict[id]!=fold for id in ID['record_id_ferc1'] ]
    is_training_data_mask = np.array(is_training_data_mask)
    
    train_set = lgb.Dataset(X.loc[is_training_data_mask], Y.loc[is_training_data_mask])
    XTest = X.loc[~is_training_data_mask]
    YTest = Y.loc[~is_training_data_mask]
    
    mod_fit = lgb.train(params=param_dict[hp_rank], train_set=train_set)
    y_fit = mod_fit.predict(XTest)
    # Test = lgb.Dataset(X.loc[~is_training_data_mask], Y.loc[~is_training_data_mask])
    
    Framework = ID[~is_training_data_mask].copy()
    Framework['y_true'] = YTest['is_match']
    Framework['y_fit'] = y_fit

    Framework = Framework[['record_id_ferc1', 'y_fit', 'y_true']]
    Framework['model_num'] = hp_rank
    Framework['fold_num'] = fold
    y_fit_list.append(Framework)

  0%|          | 0/4 [00:00<?, ?it/s]

/opt/miniconda3/lib/python3.12/site-packages/lightgbm/engine.py:204: UserWarning: Found `num_trees` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Number of positive: 5037, number of negative: 10074000
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.521521 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2907
[LightGBM] [Info] Number of data points in the train set: 10079037, number of used features: 57
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.000500 -> initscore=-7.600902
[LightGBM] [Info] Start training from score -7.600902


/opt/miniconda3/lib/python3.12/site-packages/lightgbm/engine.py:204: UserWarning: Found `num_trees` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Number of positive: 5090, number of negative: 10180000
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.642715 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2910
[LightGBM] [Info] Number of data points in the train set: 10185090, number of used features: 57
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.000500 -> initscore=-7.600902
[LightGBM] [Info] Start training from score -7.600902


/opt/miniconda3/lib/python3.12/site-packages/lightgbm/engine.py:204: UserWarning: Found `num_trees` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Number of positive: 5037, number of negative: 10074000
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.636829 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2907
[LightGBM] [Info] Number of data points in the train set: 10079037, number of used features: 57
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.000500 -> initscore=-7.600902
[LightGBM] [Info] Start training from score -7.600902


/opt/miniconda3/lib/python3.12/site-packages/lightgbm/engine.py:204: UserWarning: Found `num_trees` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Number of positive: 5090, number of negative: 10180000
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.556522 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2910
[LightGBM] [Info] Number of data points in the train set: 10185090, number of used features: 57
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.000500 -> initscore=-7.600902
[LightGBM] [Info] Start training from score -7.600902


In [99]:
AggYFit = pd.concat(y_fit_list).reset_index(drop=True)

AggYFit['groupwise_max_y_fit'] = AggYFit.groupby('record_id_ferc1')['y_fit'].transform('max')
AggYFit['y_fit_adj'] = AggYFit['y_fit'] == AggYFit['groupwise_max_y_fit']
AggYFit['y_fit_adj'] = 1 * AggYFit['y_fit_adj']
AggYFit = AggYFit[['model_num', 'fold_num', 'record_id_ferc1', 'y_true', 'y_fit', 'y_fit_adj']]
AggYFit.head()

,model_num,fold_num,record_id_ferc1,y_true,y_fit,y_fit_adj
0,0,0,f1_steam_2005_12_108_0_1,0,0.000017,0
1,0,0,f1_steam_2005_12_108_0_1,0,0.000017,0
2,0,0,f1_steam_2005_12_108_0_1,0,0.000015,0
3,0,0,f1_steam_2005_12_108_0_1,0,0.000017,0
4,0,0,f1_steam_2005_12_108_0_1,0,0.000024,0
